In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Resize TIFF**

In [ ]:
!pip install opencv-python

In [ ]:
!pip install rasterio

In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import Affine
import math

in_path = "/content/drive/MyDrive/PlantIdentification/Big/VNUF_22052024_134_modified_2.tif"
out_path = "/content/drive/MyDrive/PlantIdentification/Big/VNUF_22052024_134_modified_2_big.tif"
scale = 0.04834786752541137683/0.004499902323160890477  # 35% kích thước

with rasterio.open(in_path) as src:
    # Tính kích thước mới
    # Đảm bảo new_width và new_height là bội số của 16 cho TIFF có tiled = True
    block_size = 16
    new_width  = block_size * math.ceil((src.width  * scale) / block_size)
    new_height = block_size * math.ceil((src.height * scale) / block_size)

    # Transform mới (pixel size tăng theo 1/scale)
    a, b, c, d, e, f = src.transform[:6]
    new_transform = Affine(a / scale, b, c, d, e / scale, f)

    # Chuẩn bị profile xuất, giữ metadata
    profile = src.profile.copy()
    profile.update({
        "width": int(new_width),
        "height": int(new_height),
        "transform": new_transform,
        "compress": "lzw",
        "tiled": True,
        "BIGTIFF": "IF_SAFER"
    })

    resampling = Resampling.lanczos if src.count in (3,4) else Resampling.average

    with rasterio.open(out_path, "w", **profile) as dst:
        for i in range(1, src.count + 1):
            data = src.read(
                i,
                out_shape=(int(new_height), int(new_width)),
                resampling=resampling
            )
            dst.write(data, i)

        # Sao chép tags (metadata phi địa lý)
        dst.update_tags(**src.tags())
        for i in range(1, src.count + 1):
            dst.update_tags(i, **src.tags(i))

print("Done.")

Done.


**Convert small JSON to Bigger JSON**

In [ ]:
import rasterio

with rasterio.open("/content/drive/MyDrive/PlantIdentification/Big/VNUF_22052024_134_modified_2_big.tif") as ds:
    print("Width (pixels):", ds.width)
    print("Height (pixels):", ds.height)
    print("Pixel size (x, y):", ds.res)


Width (pixels): 83504
Height (pixels): 95488
Pixel size (x, y): (0.0040958357687858596, 0.0040958357687858596)


In [ ]:
import json
import copy

# === THÔNG SỐ ĐẦU VÀO ===
# Tỉ lệ pixel size giữa 2 ảnh (ảnh gốc / ảnh mới)
scale_x =  0.04834786752541137683 / 0.0040958357687858596
scale_y =  0.04834786752541137683 / 0.0040958357687858596

#tên ảnh đích
image_file = "/content/drive/MyDrive/PlantIdentification/Big/VNUF_22052024_134_modified_2_big.tif"

# Tọa độ tương ứng góc trên trái: (0, 0) trong ảnh 100m → (3383, 2988) trong ảnh 150m
offset_x = 0
offset_y = 0

# Kích thước ảnh đích
width_destination = 83504
height_destination = 95488

# === ĐỌC FILE GỐC ===
with open("/content/drive/MyDrive/PlantIdentification/cocojsonfolder/VNUF_22052024_134_modified_2_small.json", "r", encoding="utf-8") as f:
    coco = json.load(f)

# === TẠO BẢN SAO COCO MỚI ===
new_coco = copy.deepcopy(coco)

# Cập nhật thông tin ảnh mới
new_coco["images"] = [{
    "id": 2,
    "file_name": image_file,
    "width": width_destination,
    "height": height_destination,
    "license": 0,
    "flickr_url": "",
    "coco_url": "",
    "date_captured": 0
}]

# === HÀM CHUYỂN ĐỔI TOẠ ĐỘ ===
def transform_point(x, y):
    new_x = x * scale_x + offset_x
    new_y = y * scale_y + offset_y
    return round(new_x, 2), round(new_y, 2)

def transform_segmentation(seg):
    return [coord for x, y in zip(seg[::2], seg[1::2]) for coord in transform_point(x, y)]

def compute_bbox(seg):
    xs = seg[::2]
    ys = seg[1::2]
    x_min = min(xs)
    y_min = min(ys)
    x_max = max(xs)
    y_max = max(ys)
    return [round(x_min, 2), round(y_min, 2), round(x_max - x_min, 2), round(y_max - y_min, 2)]

# === CHUYỂN TOÀN BỘ ANNOTATION ===
for ann in new_coco["annotations"]:
    ann["image_id"] = 2  # gán sang ảnh mới

    new_seg = transform_segmentation(ann["segmentation"][0])
    ann["segmentation"] = [new_seg]

    ann["bbox"] = compute_bbox(new_seg)

# === GHI FILE MỚI ===
with open("/content/drive/MyDrive/PlantIdentification/cocojsonfolder/VNUF_22052024_134_modified_2_big.json", "w", encoding="utf-8") as f:
    json.dump(new_coco, f, ensure_ascii=False, indent=2)

print("✅ Đã chuyển đổi annotation sang ảnh mới nek :))!")


✅ Đã chuyển đổi annotation sang ảnh mới nek :))!


**Transfer to GeoJSON**

In [ ]:
import json
import geopandas as gpd
from shapely.geometry import Polygon
from osgeo import gdal
from tqdm import tqdm

print('test')

json_path = "Orthophoto\cocojsonfolder\VNUF-10082024-211m-234_modified_big.json"
tif_path = "Orthophoto\Big\VNUF_22052024_134_modified_2_big.tif"  # need provide
output_geojson = "Orthophoto\geojsonfolder\VNUF-10082024-211m-234_modified_big.geojson"

ds = gdal.Open(tif_path)
gt = ds.GetGeoTransform()

def pixel_to_geo(x, y):
    geo_x = gt[0] + x * gt[1] + y * gt[2]
    geo_y = gt[3] + x * gt[4] + y * gt[5]
    return (geo_x, geo_y)

with open(json_path, "r", encoding="utf-8") as f:
    coco = json.load(f)

categories = {cat["id"]: cat["name"] for cat in coco["categories"]}
image_info = {img["id"]: img for img in coco["images"]}
annotations = coco["annotations"]

features = []
for ann in tqdm(annotations):
    image_id = ann["image_id"]
    image_name = image_info[image_id]["file_name"]

    for seg in ann["segmentation"]:
        coords = list(zip(seg[::2], seg[1::2]))  # pixel (x, y)
        geo_coords = [pixel_to_geo(x, y) for x, y in coords]
        poly = Polygon(geo_coords)

        features.append({
            "geometry": poly,
            "properties": {
                "category_id": ann["category_id"],
                "category_name": categories[ann["category_id"]],
                "image_id": image_id,
                "image_name": image_name,
                "ann_id": ann["id"]
            }
        })

# Lưu ra GeoJSON
gdf = gpd.GeoDataFrame.from_features(features, crs="EPSG:32648")  # hoặc hệ toạ độ đúng của ảnh TIF
gdf.to_file(output_geojson, driver="GeoJSON")


<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:10: SyntaxWarning: invalid escape sequence '\B'
<>:11: SyntaxWarning: invalid escape sequence '\g'
<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:10: SyntaxWarning: invalid escape sequence '\B'
<>:11: SyntaxWarning: invalid escape sequence '\g'
/tmp/ipython-input-3580630085.py:9: SyntaxWarning: invalid escape sequence '\c'
  json_path = "Orthophoto\cocojsonfolder\VNUF-10082024-211m-234_modified_big.json"
/tmp/ipython-input-3580630085.py:10: SyntaxWarning: invalid escape sequence '\B'
  tif_path = "Orthophoto\Big\VNUF_22052024_134_modified_2_big.tif"  # need provide
/tmp/ipython-input-3580630085.py:11: SyntaxWarning: invalid escape sequence '\g'
  output_geojson = "Orthophoto\geojsonfolder\VNUF-10082024-211m-234_modified_big.geojson"


ModuleNotFoundError: No module named 'geopandas'

**Crop Tree Normal**

In [ ]:
import os
import json
import numpy as np
import cv2
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # Tắt giới hạn

print('begin processing')

coco_file = '/content/drive/MyDrive/PlantIdentification/cocojsonfolder/VNUF_22052024_134_modified_2_big.json'  # COCO file
image_file = '/content/drive/MyDrive/PlantIdentification/Big/VNUF_22052024_134_modified_2_big.tif'  # file_name in annotation
output_root = '/content/drive/MyDrive/PlantIdentification/Annotations/VNUF_22052024_134_modified_2_big_annotations'  # thư mục lưu kết quả

with open(coco_file, 'r', encoding='utf-8') as f:
    coco = json.load(f)

image_pil = Image.open(image_file).convert("RGB")
image_np = np.array(image_pil)  # shape: (H, W, 3), dtype=uint8

# Create map category_id -> tên loài ===
category_map = {cat['id']: cat['name'] for cat in coco['categories']}

# handle for each annotation
for ann in coco['annotations']:
    cat_id = ann['category_id']
    cat_name = category_map.get(cat_id, f"unknown_{cat_id}")
    segmentation = ann['segmentation'][0]  # Polygon [x1, y1, x2, y2, ..., xn, yn]
    polygon = np.array(segmentation).reshape(-1, 2).astype(np.int32)

    # create white mask for the polygon
    mask = np.zeros(image_np.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [polygon], 255)

    # Crop by bounding box of the polygon
    x, y, w, h = cv2.boundingRect(polygon)
    crop_img = image_np[y:y+h, x:x+w]
    crop_mask = mask[y:y+h, x:x+w]

    # cvt to RGBA with transparent background
    crop_rgba = np.dstack([crop_img, crop_mask])

    # crete output directory for the species
    output_dir = os.path.join(output_root, cat_name)
    os.makedirs(output_dir, exist_ok=True)

    # png with transparent background
    out_path = os.path.join(output_dir, f"{ann['id']}.png")
    Image.fromarray(crop_rgba).save(out_path)

print("Digger_OKla?")


begin processing


SystemError: tile cannot extend outside image

In [ ]:
import os
import json
import numpy as np
import cv2
import rasterio
from rasterio.windows import Window
from PIL import Image

print("Begin processing with rasterio")

coco_file = '/content/drive/MyDrive/PlantIdentification/cocojsonfolder/VNUF_22052024_134_modified_2_big.json'
image_file = '/content/drive/MyDrive/PlantIdentification/Big/VNUF_22052024_134_modified_2_big.tif'
output_root = '/content/drive/MyDrive/PlantIdentification/Annotations/VNUF_22052024_134_modified_2_big_annotations'

with open(coco_file, 'r', encoding='utf-8') as f:
    coco = json.load(f)

category_map = {cat['id']: cat['name'] for cat in coco['categories']}

with rasterio.open(image_file) as src:
    img_w, img_h = src.width, src.height

    for ann in coco['annotations']:
        cat_id = ann['category_id']
        cat_name = category_map.get(cat_id, f"unknown_{cat_id}")

        segmentation = ann['segmentation'][0]
        polygon = np.array(segmentation).reshape(-1, 2).astype(np.int32)

        # === Bounding box từ polygon ===
        x, y, w, h = cv2.boundingRect(polygon)

        # === Clamp tránh vượt biên ===
        x1 = max(0, x)
        y1 = max(0, y)
        x2 = min(img_w, x + w)
        y2 = min(img_h, y + h)

        if x2 <= x1 or y2 <= y1:
            continue  # bỏ annotation lỗi

        win_w = x2 - x1
        win_h = y2 - y1

        window = Window(x1, y1, win_w, win_h)

        # === Read image patch (RGB) ===
        patch = src.read([1, 2, 3], window=window)  # (3, H, W)
        patch = np.transpose(patch, (1, 2, 0))      # (H, W, 3)

        # === Mask theo patch ===
        poly_local = polygon - np.array([x1, y1])
        mask = np.zeros((win_h, win_w), dtype=np.uint8)
        cv2.fillPoly(mask, [poly_local], 255)

        # === RGBA ===
        rgba = np.dstack([patch, mask])

        # === Save ===
        out_dir = os.path.join(output_root, cat_name)
        os.makedirs(out_dir, exist_ok=True)

        out_path = os.path.join(out_dir, f"{ann['id']}.png")
        Image.fromarray(rgba).save(out_path)

print("Rasterio_Done_OK 👌")


Begin processing with rasterio
Rasterio_Done_OK 👌


**Split - Tran : Test : Val**

In [ ]:
import os
import shutil
import math
from PIL import Image
from tqdm import tqdm

def split_custom_ratio_sorted(input_dir, output_dir):
    classes = os.listdir(input_dir)

    for cls in tqdm(classes, desc="Processing classes"):
        cls_path = os.path.join(input_dir, cls)
        if not os.path.isdir(cls_path):
            continue

        # Lấy danh sách ảnh và tính kích thước
        image_files = []
        for img_name in os.listdir(cls_path):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
                img_path = os.path.join(cls_path, img_name)
                try:
                    with Image.open(img_path) as img:
                        width, height = img.size
                        area = width * height
                        image_files.append((img_name, area))
                except Exception as e:
                    print(f"❌ Lỗi khi đọc ảnh {img_name}: {e}")

        # Sắp xếp giảm dần theo kích thước (diện tích)
        image_files.sort(key=lambda x: x[1], reverse=True)
        images_sorted = [img[0] for img in image_files]

        total = len(images_sorted)
        if total == 1:
            train_num = 1
        else:
            train_num = math.floor(total * 0.6)
        rest = total - train_num

        val_num = math.ceil(rest * 1 / 2)
        test_num = rest - val_num

        splits = {
            'train': images_sorted[:train_num],
            'val': images_sorted[train_num:train_num + val_num],
            'test': images_sorted[train_num + val_num:]
        }

        # Copy ảnh sang thư mục output
        for split, split_imgs in splits.items():
            split_dir = os.path.join(output_dir, split, cls)
            os.makedirs(split_dir, exist_ok=True)

            for img in split_imgs:
                src = os.path.join(cls_path, img)
                dst = os.path.join(split_dir, img)
                shutil.copy2(src, dst)

    print("✅ Done splitting!")

# Ví dụ sử dụng
split_custom_ratio_sorted("/content/drive/MyDrive/PlantIdentification/Annotations/VNUF_22052024_134_modified_2_big_annotations",
                          "/content/drive/MyDrive/PlantIdentification/Annotations/VNUF_22052024_134_modified_2_big_annotations_splitted")

Processing classes: 100%|██████████| 25/25 [05:30<00:00, 13.24s/it]

✅ Done splitting!


In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np

# ==== CẤU HÌNH ====
input_dir = Path("/content/drive/MyDrive/PlantIdentification/Annotations/VNUF_22052024_134_modified_2_big_annotations_splitted/val")  # Thư mục chứa ảnh gốc
output_dir = Path("/content/drive/MyDrive/PlantIdentification/Patches/VNUF_22052024_134_modified_2_big_patches/val")  # Thư mục lưu patch

patch_size = 224
overlap_ratio = 0.2  # Overlap 20%
step = int(patch_size * (1 - overlap_ratio))  # = 44 với patch_size=224

def ensure_dir(path: Path):
    path.mkdir(parents=True, exist_ok=True)

def is_mostly_black(patch: Image.Image, threshold_ratio=0.2):
    np_patch = np.array(patch)
    # Mask các pixel hoàn toàn đen: R=G=B=0
    black_pixels = np.all(np_patch == 0, axis=-1)
    black_ratio = np.sum(black_pixels) / black_pixels.size
    return black_ratio >= threshold_ratio

def crop_image_with_overlap(img_path: Path, out_dir: Path, patch_size=224, step=179):
    img_name = img_path.stem

    try:
        img = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"❌ Không thể đọc ảnh: {img_path} ({e})")
        return

    w, h = img.size
    patch_id = 1
    for top in range(0, h - patch_size + 1, step):
        for left in range(0, w - patch_size + 1, step):
            box = (left, top, left + patch_size, top + patch_size)
            patch = img.crop(box)

            # Bỏ patch nếu >= 50% pixel là màu đen
            if is_mostly_black(patch):
                continue

            out_name = f"{img_name}_patch{patch_id}.png"
            out_path = out_dir / out_name
            try:
                patch.save(out_path)
            except Exception as e:
                print(f"❌ Không thể ghi ảnh: {out_path} ({e})")
            patch_id += 1

def process_all_images(input_dir: Path, output_dir: Path):
    for class_dir in input_dir.iterdir():
        if not class_dir.is_dir():
            continue

        out_class_dir = output_dir / class_dir.name
        ensure_dir(out_class_dir)

        for img_file in class_dir.glob("*.png"):
            crop_image_with_overlap(img_file, out_class_dir, patch_size, step)

if __name__ == "__main__":
    process_all_images(input_dir, output_dir)
    print("✅ Hoàn tất cắt patch (bỏ patch quá đen) với overlap 20%.")


✅ Hoàn tất cắt patch (bỏ patch quá đen) với overlap 20%.
